In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r "/content/drive/My Drive/NSL_KDD" NSL_KDD


In [0]:
import pandas as pd
import numpy as np
import sys
import sklearn
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]


df = pd.read_csv("NSL_KDD/KDDTrain+_2.csv", header=None, names = col_names)
df_test = pd.read_csv("NSL_KDD/KDDTest+_2.csv", header=None, names = col_names)

In [5]:
df.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [6]:
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)

Dimensions of the Training set: (125973, 42)
Dimensions of the Test set: (22543, 42)


In [7]:
df.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,125973.00000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.0,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,0.001342,0.001103,0.302192,0.012669,0.000413,0.004096,0.0,0.000008,0.009423,84.107555,27.737888,0.284485,0.282485,0.119958,0.121183,0.660928,0.063053,0.097322,182.148945,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240
std,2604.51531,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,0.036603,0.045154,24.399618,0.483935,0.022181,0.099370,0.0,0.002817,0.096612,114.508607,72.635840,0.446456,0.447022,0.320436,0.323647,0.439623,0.180314,0.259830,99.206213,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000,82.000000,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,14.000000,8.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,255.000000,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,143.000000,18.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.060000,0.000000,255.000000,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000
max,42908.00000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,1.000000,2.000000,7468.000000,43.000000,2.000000,9.000000,0.0,1.000000,1.000000,511.000000,511.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
print('Label distribution Training set:')
print(df['label'].value_counts())

Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64


In [9]:
print('Label distribution Test set:')
print(df_test['label'].value_counts())

Label distribution Test set:
normal             9710
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
phf                   2
perl                  2
sqlattack             2
udpstorm              2
loadmodule            2
worm                  2
imap                  1
Name: label, dtype: int64


In [10]:
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

Training set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'label' has 23 categories

Distribution of categories in service:
http        40338
private     21853
domain_u     9043
smtp         7313
ftp_data     6860
Name: service, dtype: int64


In [11]:
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Test set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 63 categories
Feature 'flag' has 11 categories
Feature 'label' has 38 categories


In [12]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']
categorical_columns=['protocol_type', 'service', 'flag'] 
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


In [13]:
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service`
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2

['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_harvest', 'service_hostnames', 'service_http', 'service_http_2784', 'service_http_443', 'service_http_8001', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_private', 'service_red_i', 'service_remote_job', 'serv

In [14]:
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

   protocol_type  service  flag
0              1       20     9
1              2       44     9
2              1       49     5
3              1       24     9
4              1       24     9


In [15]:
enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()

,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_harvest,service_hostnames,service_http,service_http_2784,service_http_443,service_http_8001,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,service_name,...,service_nnsp,service_nntp,service_ntp_u,service_other,service_pm_dump,service_pop_2,service_pop_3,service_printer,service_private,service_red_i,service_remote_job,service_rje,service_shell,service_smtp,service_sql_net,service_ssh,service_sunrpc,service_supdup,service_systat,service_telnet,service_tftp_u,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

['service_tftp_u',
 'service_http_8001',
 'service_urh_i',
 'service_http_2784',
 'service_harvest',
 'service_red_i',
 'service_aol']

In [17]:
for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape


(22543, 84)

In [18]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)

(125973, 123)
(22543, 123)


In [19]:
# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test
print(newdf['label'].head())

0    0
1    0
2    1
3    0
4    0
Name: label, dtype: object


In [0]:
newdf_X=newdf.drop(columns='label')
newdf_Y=newlabeldf
newdf_X_test=newdf_test.drop(columns='label')
newdf_Y_test=newlabeldf_test

In [0]:
# to_drop_DoS = [2,3,4]
# to_drop_Probe = [1,3,4]
# to_drop_R2L = [1,2,4]
# to_drop_U2R = [1,2,3]
# DoS_df=newdf[~newdf['label'].isin(to_drop_DoS)];
# Probe_df=newdf[~newdf['label'].isin(to_drop_Probe)];
# R2L_df=newdf[~newdf['label'].isin(to_drop_R2L)];
# U2R_df=newdf[~newdf['label'].isin(to_drop_U2R)];

# #test
# DoS_df_test=newdf_test[~newdf_test['label'].isin(to_drop_DoS)];
# Probe_df_test=newdf_test[~newdf_test['label'].isin(to_drop_Probe)];
# R2L_df_test=newdf_test[~newdf_test['label'].isin(to_drop_R2L)];
# U2R_df_test=newdf_test[~newdf_test['label'].isin(to_drop_U2R)];
# print('Train:')
# print('Dimensions of DoS:' ,DoS_df.shape)
# print('Dimensions of Probe:' ,Probe_df.shape)
# print('Dimensions of R2L:' ,R2L_df.shape)
# print('Dimensions of U2R:' ,U2R_df.shape)
# print('Test:')
# print('Dimensions of DoS:' ,DoS_df_test.shape)
# print('Dimensions of Probe:' ,Probe_df_test.shape)
# print('Dimensions of R2L:' ,R2L_df_test.shape)
# print('Dimensions of U2R:' ,U2R_df_test.shape)

In [0]:
# # assign X as a dataframe of feautures and Y as a series of outcome variables
# X_DoS = DoS_df.drop('label',1)
# Y_DoS = DoS_df.label
# X_Probe = Probe_df.drop('label',1)
# Y_Probe = Probe_df.label
# X_R2L = R2L_df.drop('label',1)
# Y_R2L = R2L_df.label
# X_U2R = U2R_df.drop('label',1)
# Y_U2R = U2R_df.label
# # test set
# X_DoS_test = DoS_df_test.drop('label',1)
# Y_DoS_test = DoS_df_test.label
# X_Probe_test = Probe_df_test.drop('label',1)
# Y_Probe_test = Probe_df_test.label
# X_R2L_test = R2L_df_test.drop('label',1)
# Y_R2L_test = R2L_df_test.label
# X_U2R_test = U2R_df_test.drop('label',1)
# Y_U2R_test = U2R_df_test.label

In [0]:
colNames=list(newdf_X)
colNames_test=list(newdf_X_test)

In [0]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
newdf_X = sc_x.fit_transform(newdf_X)
newdf_X_test = sc_x.transform(newdf_X_test)
# scaler = preprocessing.StandardScaler().fit(newdf_X)
# newdf_X=scaler.transform(newdf_X) 
# scaler1 = preprocessing.StandardScaler().fit(newdf_X_test)
# newdf_X_test=scaler.transform(newdf_X_test) 
# scaler2 = preprocessing.StandardScaler().fit(X_Probe)
# X_Probe=scaler2.transform(X_Probe) 
# scaler3 = preprocessing.StandardScaler().fit(X_R2L)
# X_R2L=scaler3.transform(X_R2L) 
# scaler4 = preprocessing.StandardScaler().fit(X_U2R)
# X_U2R=scaler4.transform(X_U2R) 
# # test data
# scaler5 = preprocessing.StandardScaler().fit(newdf_X_test)
# newdf_X_test=scaler5.transform(newdf_X_test) 
# scaler6 = preprocessing.StandardScaler().fit(X_Probe_test)
# X_Probe_test=scaler6.transform(X_Probe_test) 
# scaler7 = preprocessing.StandardScaler().fit(X_R2L_test)
# X_R2L_test=scaler7.transform(X_R2L_test) 
# scaler8 = preprocessing.StandardScaler().fit(X_U2R_test)
# X_U2R_test=scaler8.transform(X_U2R_test)

In [0]:
# print(newdf_X.std(axis=0))

In [0]:
# X_Probe.std(axis=0);
# X_R2L.std(axis=0);
# X_U2R.std(axis=0);

In [0]:
# from sklearn.feature_selection import SelectPercentile, f_classif
# np.seterr(divide='ignore', invalid='ignore');
# selector=SelectPercentile(f_classif, percentile=10)
# X_newDoS = selector.fit_transform(X_DoS,Y_DoS)
# X_newDoS.shape

In [0]:
# true=selector.get_support()
# newcolindex_DoS=[i for i, x in enumerate(true) if x]
# newcolname_DoS=list( colNames[i] for i in newcolindex_DoS )
# newcolname_DoS

In [0]:

# X_newProbe = selector.fit_transform(X_Probe,Y_Probe)
# X_newProbe.shape

In [0]:

# true=selector.get_support()
# newcolindex_Probe=[i for i, x in enumerate(true) if x]
# newcolname_Probe=list( colNames[i] for i in newcolindex_Probe )
# newcolname_Probe

In [0]:
# X_newR2L = selector.fit_transform(X_R2L,Y_R2L)
# X_newR2L.shape

In [0]:
# true=selector.get_support()
# newcolindex_R2L=[i for i, x in enumerate(true) if x]
# newcolname_R2L=list( colNames[i] for i in newcolindex_R2L)
# newcolname_R2L

In [0]:
# X_newU2R = selector.fit_transform(X_U2R,Y_U2R)
# X_newU2R.shape

In [0]:
# true=selector.get_support()
# newcolindex_U2R=[i for i, x in enumerate(true) if x]
# newcolname_U2R=list( colNames[i] for i in newcolindex_U2R)
# newcolname_U2R

In [0]:
# print('Features selected for DoS:',newcolname_DoS)
# print()
# print('Features selected for Probe:',newcolname_Probe)
# print()
# print('Features selected for R2L:',newcolname_R2L)
# print()
# print('Features selected for U2R:',newcolname_U2R)

In [0]:
newdf_Y=newdf_Y.astype('int')
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(random_state=0)
rfe = RFE(estimator=clf1, n_features_to_select=13, step=1)
rfe.fit(newdf_X, newdf_Y)
X_rfe=rfe.transform(newdf_X)
true=rfe.support_
rfecolindex_X=[i for i, x in enumerate(true) if x]
rfecolname_X=list( colNames[i] for i in rfecolindex_X)

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators = 50)
rfe_2 = RFE(estimator=clf2, n_features_to_select=13, step=1)
rfe_2.fit(newdf_X, newdf_Y)
X_rfe_rand=rfe_2.transform(newdf_X)
true=rfe_2.support_
rfecolindex_rand=[i for i, x in enumerate(true) if x]
rfecolname_rand=list( colNames[i] for i in rfecolindex_rand)

In [0]:
# Y_Probe = Y_Probe.astype('int')
# rfe.fit(X_Probe, Y_Probe)
# X_rfeProbe=rfe.transform(X_Probe)
# true=rfe.support_
# rfecolindex_Probe=[i for i, x in enumerate(true) if x]
# rfecolname_Probe=list(colNames[i] for i in C)

In [0]:
# Y_R2L = Y_R2L.astype('int')
# rfe.fit(X_R2L, Y_R2L)
# X_rfeR2L=rfe.transform(X_R2L)
# true=rfe.support_
# rfecolindex_R2L=[i for i, x in enumerate(true) if x]
# rfecolname_R2L=list(colNames[i] for i in rfecolindex_R2L)

In [0]:
# Y_U2R = Y_U2R.astype('int')
# rfe.fit(X_U2R, Y_U2R)
# X_rfeU2R=rfe.transform(X_U2R)
# true=rfe.support_
# rfecolindex_U2R=[i for i, x in enumerate(true) if x]
# rfecolname_U2R=list(colNames[i] for i in rfecolindex_U2R)

In [0]:
print('Features selected are:',rfecolname_X)

In [0]:
 print(X_rfe.shape)
# print(X_rfeProbe.shape)
# print(X_rfeR2L.shape)
# print(X_rfeU2R.shape)

In [0]:
# clf_DoS=DecisionTreeClassifier(random_state=0)
# clf_Probe=DecisionTreeClassifier(random_state=0)
# clf_R2L=DecisionTreeClassifier(random_state=0)
# clf_U2R=DecisionTreeClassifier(random_state=0)
# clf_DoS.fit(X_DoS, Y_DoS)
# clf_Probe.fit(X_Probe, Y_Probe)
# clf_R2L.fit(X_R2L, Y_R2L)
# clf_U2R.fit(X_U2R, Y_U2R)

In [0]:
# clf_rfeDoS=DecisionTreeClassifier(random_state=0)
# clf_rfeProbe=DecisionTreeClassifier(random_state=0)
# clf_rfeR2L=DecisionTreeClassifier(random_state=0)
# clf_rfeU2R=DecisionTreeClassifier(random_state=0)
# clf_rfeDoS.fit(X_rfeDoS, Y_DoS)
# clf_rfeProbe.fit(X_rfeProbe, Y_Probe)
# clf_rfeR2L.fit(X_rfeR2L, Y_R2L)
# clf_rfeU2R.fit(X_rfeU2R, Y_U2R)

In [0]:
# clf_DoS.predict(X_DoS_test)

In [0]:
# clf_DoS.predict_proba(X_DoS_test)[0:10]

In [0]:
# Y_DoS_pred=clf_DoS.predict(X_DoS_test)
# # Create confusion matrix
# pd.crosstab(Y_DoS_test, Y_DoS_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

In [0]:
# Y_Probe_pred=clf_Probe.predict(X_Probe_test)
# # Create confusion matrix
# pd.crosstab(Y_Probe_test, Y_Probe_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

In [0]:
# Y_R2L_pred=clf_R2L.predict(X_R2L_test)
# # Create confusion matrix
# pd.crosstab(Y_R2L_test, Y_R2L_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

In [0]:
# Y_U2R_pred=clf_U2R.predict(X_U2R_test)
# # Create confusion matrix
# pd.crosstab(Y_U2R_test, Y_U2R_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

In [0]:
# from sklearn.model_selection import cross_val_score
# from sklearn import metrics
# accuracy = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='accuracy')
# print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))
# precision = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='precision')
# print("Precision: %0.5f (+/- %0.5f)" % (precision.mean(), precision.std() * 2))
# recall = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='recall')
# print("Recall: %0.5f (+/- %0.5f)" % (recall.mean(), recall.std() * 2))
# f = cross_val_score(clf_DoS, X_DoS_test, Y_DoS_test, cv=10, scoring='f1')
# print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))

In [0]:
# accuracy = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='accuracy')
# print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))
# precision = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='precision_macro')
# print("Precision: %0.5f (+/- %0.5f)" % (precision.mean(), precision.std() * 2))
# recall = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='recall_macro')
# print("Recall: %0.5f (+/- %0.5f)" % (recall.mean(), recall.std() * 2))
# f = cross_val_score(clf_Probe, X_Probe_test, Y_Probe_test, cv=10, scoring='f1_macro')
# print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))

In [0]:
# accuracy = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='accuracy')
# print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))
# precision = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='precision_macro')
# print("Precision: %0.5f (+/- %0.5f)" % (precision.mean(), precision.std() * 2))
# recall = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='recall_macro')
# print("Recall: %0.5f (+/- %0.5f)" % (recall.mean(), recall.std() * 2))
# f = cross_val_score(clf_R2L, X_R2L_test, Y_R2L_test, cv=10, scoring='f1_macro')
# print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))

In [0]:
# accuracy = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='accuracy')
# print("Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))
# precision = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='precision_macro')
# print("Precision: %0.5f (+/- %0.5f)" % (precision.mean(), precision.std() * 2))
# recall = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='recall_macro')
# print("Recall: %0.5f (+/- %0.5f)" % (recall.mean(), recall.std() * 2))
# f = cross_val_score(clf_U2R, X_U2R_test, Y_U2R_test, cv=10, scoring='f1_macro')
# print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))

In [0]:
newdf_Y=newdf_Y.astype('int')
from sklearn.tree import DecisionTreeClassifier
clf_all=DecisionTreeClassifier(random_state=0)
clf_all.fit(newdf_X, newdf_Y)

In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 50)
model.fit(newdf_X, newdf_Y)

In [0]:
model_rfe=RandomForestClassifier(n_estimators = 50)
model_rfe.fit(X_rfe_rand, newdf_Y)

In [0]:
X_test2=newdf_X_test[:,rfecolindex_rand]
model_rfe.score(X_test2, newdf_Y_test)

In [0]:

clf_rfe=DecisionTreeClassifier(random_state=0)
clf_rfe.fit(X_rfe, newdf_Y)

In [0]:
from sklearn.metrics import accuracy_score
X_test3=newdf_X_test[:,rfecolindex_X]
clf_rfe_y_pred=clf_rfe.predict(X_test3)
accuracy_score(newdf_Y_test, clf_rfe_y_pred)

In [0]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2','class 3','class 4']
classification_report(newdf_Y_test, clf_rfe_y_pred, target_names=target_names)
newdf_Y_test, clf_rfe_y_pred

In [0]:
clf_rfe.score(X_test3, newdf_Y_test)

In [0]:
clf_all_pred=clf_all.predict(newdf_X_test)


In [0]:
pd.crosstab(newdf_Y_test, clf_all_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])


In [0]:
clf_all.score(newdf_X_test, newdf_Y_test)

In [0]:
model.score(newdf_X_test, newdf_Y_test)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(newdf_X, newdf_Y)


In [0]:
knn.score(newdf_X_test, newdf_Y_test)

In [0]:
from sklearn import svm
svmmodel = svm.SVC(decision_function_shape='ovo')
svmmodel.fit(newdf_X, newdf_Y)

In [0]:
svmmodel.score(newdf_X_test, newdf_Y_test)

In [0]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(newdf_X, newdf_Y)

In [0]:
gnb.score(newdf_X_test, newdf_Y_test)